## Data preparation and cleaning

In [ ]:
import os
import cv2 
import itertools
import numpy as np 
import pandas as pd 

test_dir = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/"
train_dir = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/"

data = []

We define the input paths for the Kaggle dataset and the output path for the trained model.
We also initialize an empty list to store the data.

In [ ]:
for dirtrain in os.listdir(train_dir): 
    print(dirtrain)
    for tr in os.listdir(train_dir + dirtrain):
        img = cv2.imread(train_dir + dirtrain + "/" + tr)
        img = cv2.resize(img, (32, 32))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.reshape(32, 32, 1)
        
        data.append([img, dirtrain])        

In each iteration we:
- read image
- resize the image to matrix of 32x32
- grayscaling image
- reshape the image to 3D with one color channel (Gray)
- append each image with label to the data list

In [ ]:
for dirtest in  os.listdir(test_dir):    
    print(dirtest)
    for ts in os.listdir(test_dir + dirtest):
        img = cv2.imread(test_dir + dirtest + "/" + ts)
        img = cv2.resize(img, (32, 32))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.reshape(32, 32, 1)
        
        data.append([img, dirtest])

We do the same for the test data.

In [ ]:
import random
random.seed(20)

In [ ]:
random.shuffle(data)

We then shuffle the data randomly.

In [ ]:
x, y = [], []
for e in data:
    x.append(e[0])
    y.append(e[1])

We split the data to images in a list and labels in another list.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

x = np.array(x)
y = np.array(y)
y = y.reshape(y.shape[0],1)
enc = OneHotEncoder(handle_unknown='ignore').fit(y)
print(enc.categories_)
y = enc.transform(y).toarray()
print(f'Data   :   {str(x.shape)}')
print(f'Labels :   {str(y.shape)}')

We encode the labels using one-hot encoding. The encoder creates a new column for each label and assigns a 1 or 0 to the column. This is to avoid the model to assume a natural ordering between categories.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.2)

We then split the data into training and validation sets. We use 80% of the data for training and 20% for validation. 

In [ ]:
import tensorflow as tf

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

Activate the GPU to accelerate the training process. We are running the notebook on Kaggle for training and prediction so the GPU are listed from Kaggle.

## Building the Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (4, 4), padding='same', activation=tf.nn.relu, 
    input_shape=(32, 32, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)), Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    Dropout(0.3),

    tf.keras.layers.Conv2D(128, (2,2), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    Dropout(0.3),

    tf.keras.layers.Conv2D(256, (2,2), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    Dropout(0.3),
                    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(4,  activation=tf.nn.softmax)
])

In the above cell we define the model architecture. We use the Sequential model from Keras. The model is a stack of layers where each layer has exactly one input tensor and one output tensor. We use the following layers:
- Conv2D: 2D convolution layer (e.g. spatial convolution over images).
- MaxPooling2D: Max pooling operation for spatial data.

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
)

We now plot the model architecture. We can see the layers and the number of parameters in each layer. The number of parameters is the number of weights in each layer. The weights are the values that the model learns during training.

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
hist = model.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=64,verbose=1,shuffle=True)

We start training the model with an epoch of 100. We use the Adam optimizer and the categorical crossentropy loss function. We also use the accuracy metric to measure the performance of the model.

## Plotting the training and validation accuracy

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

fig = plt.figure(figsize=(12, 6), dpi=80)
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

We plot the loss of training. We can see that the loss is decreasing with each epoch. We can also see that the validation loss is decreasing with each epoch. This means that the model is not overfitting the training data.

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=80)
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

We then plot the accuracy of training. We can see that the accuracy is increasing with each epoch.

## Evaluating the model

In [ ]:
loss_and_metrics = model.evaluate(x_test, y_test, verbose=2)
y_pred = model.predict(x_test).argmax(axis=1)
print(f'Test Loss     : {loss_and_metrics[0]}')
print(f'Test Accuracy : {loss_and_metrics[1]}')

We predict the labels of the validation set. We then calculate the accuracy of the model by comparing the predicted labels with the actual labels and print the accuracy and loss of the model.

In [ ]:
from sklearn import metrics

df = pd.DataFrame(
  data = metrics.confusion_matrix(np.argmax(y_test, axis=1), y_pred),
  columns = ['0', '1', '2', '3'],
  index = ['0', '1', '2', '3']
)
df

In [ ]:
import seaborn as sns

sns.heatmap(df, cmap="Reds", annot=True, fmt='.0f')

We then plot the confusion matrix. The confusion matrix is a table that is often used to describe the performance of a classification model on a set of test data for which the true values are known. The confusion matrix itself is relatively simple to understand, but the related terminology can be confusing.